# TensorFlow and TensorBoard with Evaluation



## Purpose

The purpose of this lab is twofold.  

1.   to review using `TensorFlow` for modeling and evaluation with neural networks
2.   to learn about [`TensorBoard`](https://www.tensorflow.org/tensorboard)

`TensorBoard` is `TensorFlow`'s visualization toolkit, so it is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. 

We'll be using the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [1]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from keras.layers import Dense
from keras.models import Sequential
from keras.losses import binary_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

2022-10-28 20:23:26.145831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-28 20:23:26.145864: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Loading the data

In [57]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

It is your choice how much or how little EDA that you perform. But you should do enough EDA that you feel comfortable with the data and what you'll need to do to make it so that you can run a neural network on it.

It is prudent to investigate the attributes of the data frames, create visualizations, and perform data analysis.

### Preprocessing

Here are some suggestions:

*   Check to see if you have missing data in the train and test sets.
*   Combine the test and train sets into a dataframe called *combined* since it will make preprocessing more efficient
*   Use the median of the column to replace missing data for numeric values
*   Use the mode of the column to replace missing data for categorical values
*   Change *Pclass* from 1, 2, 3, to 1st, 2nd, 3rd
*   Create a new variable *Child*, where you use the *Age* column to assign those who are 18 years or older a 1, and those younger a 0
*   For *Sex*, let Male = 0 and Female = 1
*   For *Embarked*, C=0, Q=1, and S=2
*   The names of the passengers are not meaningful for the model, but their titles may be.
  * Use the **Code Chunk 1** below to help you with this.
*   Drop the irrelevant columns: *PassengerId*, *Name*, *Ticket*, *Cabin* and use `get_dummies`
  * Use the **Code Chunk 2** below to help you with this.
*  Resplit and scale the data.
  * Use the **Code Chunk 3** below to help you with this.

Of course, for the two code chunks you need to uncomment the germane lines of code.

In [58]:
combined = pd.concat([train, test])
combined.info()

fill_values = {'Age': combined.Age.median(),
               'Fare': combined.Fare.median(),
               'Embarked': combined.Embarked.mode()
                }

cleaned_df = combined.fillna(fill_values, axis=0)
cleaned_df.loc[combined.Embarked.isna(), ['Embarked']] = 'S' # the fillna did not effectively replace these
cleaned_df.Pclass.replace({1: '1st', 2: '2nd', 3: '3rd'}, inplace=True)

def is_child(row):
    if(row['Age'] >= 18): return 1
    else: return 0

cleaned_df['Child'] = cleaned_df.apply(is_child, axis=1)
cleaned_df.Sex.replace({'male': 0, 'female':1}, inplace=True)
cleaned_df.Embarked.replace({'C': 0, 'Q': 1, 'S': 2}, inplace=True)

cleaned_df.info()
combined = cleaned_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survi

In [47]:
cleaned_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
0,1,0.0,3rd,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2,1
1,2,1.0,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0,1
2,3,1.0,3rd,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,1
3,4,1.0,1st,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2,1
4,5,0.0,3rd,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2,1


**Code Chunk 1** Titles

In [59]:
# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
   for j in range(len(english_titles)):
       if i == j:
           combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

**Code Chunk 2** Dropping and Dummies

In [60]:
#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], 1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

/tmp/ipykernel_61/3435163405.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  combined.drop(['PassengerId','Name','Ticket','Cabin'], 1, inplace=True)


**Code Chunk 3** Resplitting and scalling the data

In [66]:
# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

/tmp/ipykernel_61/2204541156.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = train.drop(['Survived'],1)


## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

In [69]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=10,kernel_initializer='uniform',activation='relu',input_dim=13))
    classifier.add(Dense(units=128,kernel_initializer='uniform',activation='relu'))
    classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

# Note: Do not use regularization methods or GridSearch. Those will be for next time!

#### Fitting the optimal model and evaluating with `TensorBoaard`

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [70]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

rm: cannot remove 'logs/fit/20221028-211448/train': Directory not empty


#### Fitting the optimal model and evaluating with `TensorBoaard`

In [71]:
# Using KerasClassifier

# Replace the optimizer, batch_size, and epochs appropriately
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer='Adam',
                             batch_size=10,
                             epochs=16)

# Fit the model with the tensorboard_callback

classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

/tmp/ipykernel_61/1378190792.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  classifier = KerasClassifier(build_fn = build_classifier,


Epoch 1/16
90/90 [==============================] - 1s 2ms/step - loss: 0.6293 - accuracy: 0.7217
Epoch 2/16
90/90 [==============================] - 0s 1ms/step - loss: 0.4546 - accuracy: 0.8159
Epoch 3/16
90/90 [==============================] - 0s 1ms/step - loss: 0.4188 - accuracy: 0.8260
Epoch 4/16
90/90 [==============================] - 0s 1ms/step - loss: 0.4110 - accuracy: 0.8249
Epoch 5/16
90/90 [==============================] - 0s 1ms/step - loss: 0.4074 - accuracy: 0.8283
Epoch 6/16
90/90 [==============================] - 0s 1ms/step - loss: 0.4023 - accuracy: 0.8316
Epoch 7/16
90/90 [==============================] - 0s 1ms/step - loss: 0.3988 - accuracy: 0.8384
Epoch 8/16
90/90 [==============================] - 0s 1ms/step - loss: 0.3990 - accuracy: 0.8316
Epoch 9/16
90/90 [==============================] - 0s 1ms/step - loss: 0.3951 - accuracy: 0.8395
Epoch 10/16
90/90 [==============================] - 0s 1ms/step - loss: 0.3988 - accuracy: 0.8361
Epoch 11/16
90/90 [

In [74]:
# Call TensorBoard
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 209), started 0:00:34 ago. (Use '!kill 209' to kill it.)

#### Results and Predictions

In [73]:
# This will export your predictions to a .csv. Uncomment.

preds = classifier.predict(test)
results = ids.assign(Survived=preds)
results['Survived'] = results['Survived'].apply(lambda row: 1 if row > 0.5 else 0)
results.to_csv('titanic_submission.csv',index=False)
results.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

We'll continue with this for the next lesson when we learn about model regularization.